<a href="https://colab.research.google.com/github/nprimavera/MPC/blob/main/Project3_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MECS6616 Spring 2025 - Project 3**

# **Introduction**

***IMPORTANT:***
- **Before starting, make sure to read the [Assignment Instructions](https://courseworks2.columbia.edu/courses/215046/pages/assignment-instructions) page on Courseworks to understand the workflow and submission requirements for this project.**

**FOR PROJECT 3!!!**
- Apart from the link to your notebook, you are also required to submit your collected data `.pkl` files and your chosen model checkpoint `.pth` files to Coursework. You will have two files each for parts 2 and 3 for a total of 4 files.
- Your part 2 files should be named `data_pt2.pkl` and `dynamics_pt2.pth`.
- Your part 3 files should be named `data_pt3.pkl` and `dynamics_pt3.pth`.
- You should put the link to your notebook in the "Comment" section of your submission.

## **Project Setup (do NOT change)**

***IMPORTANT:***
- Do NOT change this "*Project Setup*" section
- Do NOT install any other dependencies or a different version of an already provided package. You may, however, import other packages


In [1]:
# DO NOT CHANGE

# After running this cell, the folder 'mecs6616_sp25_project3' will show up in the file explorer on the left (click on the folder icon if it's not open)
# It may take a few seconds to appear
!git clone https://github.com/roamlab/mecs6616_sp25_project3.git

Cloning into 'mecs6616_sp25_project3'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 30 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 49.76 KiB | 4.52 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
# DO NOT CHANGE

# move all needed files into the working directory. This is simply to make accessing files easier
!mv /content/mecs6616_sp25_project3/* /content/

In [3]:
# Using ray for data collection is optional
!pip install ray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 MB 8.9 MB/s eta 0:00:00


# Starter Code Explanation

This project uses a simulated robot provided by the `Robot` class in `robot.py` file. Each robot is initialized with a corresponding forward dynamics (the base class for forward dynamics definition is in `arm_dynamic_base.py`). The arm_teacher is initialized with the provided ground truth forward dynamics, as defined in `arm_dynamics_teacher.py`. You are welcome to look in-depth into this file to understand how the ground truth forward dynamics is computed for an arm, given its number of links, link mass, and viscous friction of the environment - this is recommended but not necessary to successfully complete this assignment.

The Robot class inside `robot.py` which provides the interface for controlling the robot arm i.e it provides you with some functions to set/get the state and set the action for the arm and take a step using the `Robot.advance()` method. The state of the arm is a 2n-dimensional vector: n joint positions [rad] + n joint velocities [rad/s] and the action is defined as the n torques (in N-m) applied to n joints respectively.

In addition to `arm_dynamics_teacher.py` which contains the ground truth forward dynamics, you will use `arm_dynamics_student.py`, the student dynamics which internally uses a neural network model.

# Part 1: Implement Model Predictive Control



You will implement this controller by completing the MPC class. Specifically, you will implement the compute_action() method by following the algorithm discussed in the lecture. As with previous projects you are free to implement additional methods as needed or change the initialization if need be. While scoring your controller, you will be creating an instance of the MPC class and passing it to the scoring function so ensure that the arguments to the compute_action method remain the same.

Although you do not need to understand how the ArmDynamicsTeacher class works, you could use the compute_fk() method from the class. This will allow you to convert from the state value (represented as array of shape (2*n, 1) where n is num_links) to final end effector position (x, y position of the end effector). Similary we can also compute the velocity of the end effector with the code below:
```
pos_ee = dynamics.compute_fk(state)
vel_ee = dynamics.compute_vel_ee(state)
```


In [9]:
import numpy as np
from collections import defaultdict
from scipy.optimize import minimize

class MPC:

    def __init__(self, control_horizon=10, alpha=1.0, beta=0.1):
        self.control_horizon = control_horizon  # Control horizon (number of time steps for optimization)
        self.alpha = alpha  # Weight for position tracking in the cost function
        self.beta = beta  # Weight for acceleration (control effort) in the cost function

    def dynamics_model(self, state, action):

        """
        Forward model that computes the next state (position, velocity) based on the current state and action.
        Use numerical integration (e.g., Euler method or Runge-Kutta) to simulate the system.
        """

        # Simple Euler integration: q(k+1) = q(k) + dq(k) * dt and dq(k+1) = dq(k) + ddq(k) * dt
        q, dq = state
        ddq = action  # Acceleration is the control input

        dt = 0.01  # Time step for numerical integration

        # Simple Euler integration to compute next state
        q_next = q + dq * dt  # position
        dq_next = dq + ddq * dt # velocity

        return np.array([q_next, dq_next])

    def cost_function(self, predicted_states, actions, goal_state):

        """
        Compute the cost function over the predicted trajectory.
        Penalize the deviation of the predicted joint positions and velocities from the goal state, as well as the control effort (joint accelerations).
        """

        cost = 0

        for t in range(self.control_horizon):
            predicted_state = predicted_states[t]
            action = actions[t]

            # Penalty for deviation from the goal position and velocity (tracking error)
            position_error = np.linalg.norm(predicted_state[0] - goal_state[0])
            velocity_error = np.linalg.norm(predicted_state[1] - goal_state[1])

            # Penalty for control effort (acceleration)
            control_effort = np.linalg.norm(action)

            # Add to total cost
            cost += self.alpha * (position_error + velocity_error) + self.beta * control_effort

        return cost

    def compute_action(self, dynamics, state, goal, action):

        """
        Compute the optimal joint accelerations over the control horizon using MPC.
        This is the core of the MPC algorithm, where we optimize the control actions.
        """

        # Initialize action sequence and state trajectory
        action_sequence = np.zeros((self.control_horizon, action.shape[0]))  # Placeholder for action sequence
        predicted_states = np.zeros((self.control_horizon + 1, state.shape[0]))  # Placeholder for predicted states
        predicted_states[0] = state

        # "Roll out" the trajectory for the initial guess of actions
        for t in range(self.control_horizon):
            predicted_states[t + 1] = self.dynamics_model(predicted_states[t], action_sequence[t])

        # Define the optimization problem: minimize the cost function
        def objective(actions):
            # Reshape actions into a sequence of actions
            action_sequence = actions.reshape(self.control_horizon, -1)
            predicted_states = np.zeros((self.control_horizon + 1, state.shape[0]))
            predicted_states[0] = state

            # "Roll out" the trajectory for the given action sequence
            for t in range(self.control_horizon):
                predicted_states[t + 1] = self.dynamics_model(predicted_states[t], action_sequence[t])

            # Compute the total cost
            return self.cost_function(predicted_states, action_sequence, goal)

        # Use an optimization method (e.g., Sequential Least Squares Quadratic Programming) to minimize the cost
        result = minimize(objective, action.flatten(), method='SLSQP')

        # Extract the optimized action sequence
        optimal_action_sequence = result.x.reshape(self.control_horizon, -1)

        # Execute the first action from the optimal sequence and return it
        best_action = optimal_action_sequence[0]
        return best_action

## Manually testing the controller
This part is for you to manually check the performance of your controller before you are ready for it be evaluated by our scoring function.
To test your implementation run the following code. Feel free to play around with the cell or change the num_links / goal positions . You can define your controller however you would like to and then switch on gui to see how close your end effectors get to the goal position

Every time step within the environment is 0.01s, which is defined in the dynamics as `dt`.

The MPC class has a `control_horizon` variable which represents the frequency at which `controller.compute_action()` will be called

In the scoring function you will be evaluated on the distance of your end effector to the goal position and the velocity of the end effector.

In [10]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time
import math
np.set_printoptions(suppress=True)

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False

if gui:
  renderer = Renderer()
  time.sleep(1)

# Controller
controller = MPC()

# Resetting the arm will set its state so that it is in the vertical position,
# and set the action to be zeros
arm.reset()

# Choose the goal position you would like to see the performance of your controller
goal = np.zeros((2, 1))
goal[0, 0] = 2.5
goal[1, 0] = -0.7
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)

# Initial action (start with zeros)
action = np.zeros((arm.dynamics.get_action_dim(), 1))

# Control loop
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:

    # Get the current state of the arm
    state = arm.get_state()

    # Measuring distance and velocity of end effector
    pos_ee = dynamics_teacher.compute_fk(state)
    dist = np.linalg.norm(goal-pos_ee)
    vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))
    print(f'At timestep {s}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')

    # Compute the optimal action using the MPC controller
    action = controller.compute_action(arm.dynamics, state, goal, action)

    # Apply the action to the arm
    arm.set_action(action)

At timestep 0: Distance to goal: 3.3970575502926055, Velocity of end effector: 5.539171676143414e-18


ValueError: could not broadcast input array from shape (6,1) into shape (6,)

## Grading and Evaluation for Part 1
Your controller will be graded on 6 tests. 2 tests each for the 1-link ,2-link, and 3-link arms. The arm will start off in the initial state with the arms pointing stright down. The testing criteria depend on the distance and the velocity of the end effectors . Each test will run the robot arm for **5.0 seconds**. At the end of the 5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.1 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`

After all of the tests, your score is summed up and then scaled out of 5 points.


In [ ]:
# Scoring using score_mpc
controller = MPC()
gui = False

In [ ]:
# DO NOT CHANGE
score_mpc_true_dynamics(controller, gui)

# Part 2: Learning Forward Dynamics

### Background Information

From here on, you will only be working with a 2-link arm.

In part 2, you will be training a model that learns the ground truth dynamics of a 2-link arm. This part uses two 2-link arms, one called arm_teacher (blue) and the other called arm_student (red), as shown in the image below. For each test, a torque will be applied to the first joint of both arms for 5 seconds. arm_teacher is moving according to the provided ground truth forward dynamics and your job is to use deep learning to train the arm_student to learn the forward dynamics of arm_teacher so that it can imitate its behavior. The forward dynamics is a function that takes in the current state of an action applied to the arm, and then computes the new state of the arm. This part uses a time step of 0.01 second, meaning each time we advance the simulation, we compute the forward dynamics for 0.01 second. In the example image, the student arm is not updating its state and remains static but we will make it move after training is done.

<div>
<img src="https://github.com/roamlab/mecs6616_sp25_project3/blob/main/imgs/example.png?raw=true" width="600"/>
</div>

The state of each arm is defined with a (4,1)-dimensional numpy array (two joint positions in radians + two joint velocities in radians per second). An action is defined as the two torques (in Nm) applied to the two joints respectively, which is a (2,1) numpy array. **Throughout this part, we make the problem simpler by only applying a torque to the first joint, so the actions always look like `[torque,0]`.** Also, when scoring your model the robot will always start off in a hanging position, meaning an initial state of `[-pi/2,0,0,0]` so if the collected data from part 2.1 looks similar, the model will perform better. The `robot.py` file provides you with some functions to set/get the state and set the action for the arm. Make sure you understand `robot.py` well enough before getting started.

`geometry.py` provides some geometry functions, `render.py` defines how the visualization is rendered, `custom_plot.py` provides data visualization functionality. These three files are not of particular interest for completing this part.

## Part 2.1: Collecting Data

You will first need to complete the cell below to collect a dataset for training the forward dynamics. After running the cell, it should generate a pickle file `data_pt2.pkl` that contains a data dictionary `data = {'X': X, 'Y': Y}`. The shape of `data['X']` should be (`num_samples`, 6), the first 4 elements are state and the last 2 elements are the action. The shape of `data['Y']` should be (`num_samples`, 4), which saves the next state after applying the action using the ground truth forward dynamics of arm_teacher. You are responsible for deciding and collecting an appropriate amount of samples. Thus, num_samples is entirely up to you and your model.

**After the data file is generated, `data_pt2.pkl` should appear under the 'Files' icon in the left sidebar. You can download this file by right clicking the file name. You are required to submit this file. Please do not change its name.**

In the cell below, we have provided a minimal example of simulating the arm_teacher for 5 seconds. The GUI visualization is turned on and you should see the behavior of arm_teacher. The visualization can drastically slow down the simulator and you should turn it off when collecting a large amount of data.

In [ ]:
import numpy as np
import os
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
import pickle
import math
from render import Renderer
import time
from custom_plot import plot_positions_velocities_with_fixed_bounds

# DO NOT CHANGE
# Teacher arm
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01
)
arm_teacher = Robot(dynamics_teacher)

# ---
# You code starts here. X and Y should eventually be populated with your collected data
# Control the arm to collect a dataset for training the forward dynamics.
X = np.zeros((0, arm_teacher.dynamics.get_state_dim() + arm_teacher.dynamics.get_action_dim()))
Y = np.zeros((0, arm_teacher.dynamics.get_state_dim()))

# We run the simulator for 5 seconds with a time step of 0.01 second,
# so there are 500 steps in total
num_steps = 500

# GUI visualization, this will drastically reudce the speed of the simulator!
# Set this to false once you understand how the code works
gui = True

# Define the initial state of the robot, such that it is vertical
initial_state = np.zeros((arm_teacher.dynamics.get_state_dim(), 1))  # position and velocity
initial_state[0] = -math.pi / 2.0

# Set the initial state of the arm. Input to set_state() should be of shape (6, 1)
arm_teacher.set_state(initial_state)

# Define the action, applying 1Nm torque to the first joint
action = np.zeros((arm_teacher.dynamics.get_action_dim(), 1))
action[0] = 1

# Set the action. Input to set_action() should be of shape (3, 1)
arm_teacher.set_action(action)

# Initialize the GUI
if gui:
    renderer = Renderer()
    time.sleep(1)

for s in range(num_steps):
    # Get the current state
    state = arm_teacher.get_state()

    # The advance function will simulate the action for 1 time step
    arm_teacher.advance()
    if gui:
        renderer.plot([(arm_teacher, 'tab:blue')])

    # Get the new state after advancing one time step
    new_state = arm_teacher.get_state()
# ---

# DO NOT CHANGE
# Save the collected data in the data_pt2.pkl file
data = {'X': X, 'Y': Y}
pickle.dump(data, open( "data_pt2.pkl", "wb" ) )

Once you've stored X and Y in data, you may uncomment the cell below to see the distribution of end-effector positions and velocities

In [ ]:
# Plot the distribution of collected samples
# plot_positions_velocities_with_fixed_bounds(data['Y'], [1,1])

## Part 2.2: Learning Forward Dynamics

### Training

After the data is collected, you will then need to complete the cell below to use the collected dataset to learn the forward dynamics.

The code already creates the dataset class and loads the dataset with a random 0.8/0.2 train/test split for you. This cell should save the model that it trains. You should use a specific procedure for saving, outlined below. Note that the saving code is not already included so you will have to add it yourself.

In machine learning, it is a very good practice to save not only the final model but also the checkpoints, such that you have a wider range of models to choose from. We provide a code snippet for you and for each epoch of your training, you should use it to save the model at that epoch.

```
model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
if not os.path.exists(os.path.join(model_dir, model_folder_name)):
    os.makedirs(os.path.join(model_dir, model_folder_name))
torch.save(model.state_dict(), os.path.join(model_dir, model_folder_name, 'dynamics_pt2.pth'))
```

The output from running this code should be a folder as below:

```
models/
    2025-03-08_23-57-50/
        epoch_0001_loss_0.00032930/
            dynamics_pt2.pth
        epoch_0002_loss_0.00009413/
            dynamics_pt2.pth   
        ...  
```

You can see that every time you run this cell, a folder whose name is the time you started will be created under `models`. Checkpoints from all epochs will be saved and then the folder name for saving the checkpoint indicates the epoch number and loss on the holdout test set. Recording checkpoints this way allows you to easily pick the model with the smallest loss.

### Important: choosing the best model

Your code should keep track of the checkpoint with the smallest loss on the test set. You should save the path of that checkpoint to the variable `model_path`. An example value of `model_path` could be `models/2025-03-07_20-14-32/epoch_0046_loss_0.00000005/dynamics_pt2.pth`. In the evaluation code, the checkpoint from `model_path` will be loaded and evaluated.

You should also download the `dynamics_pt2.pth` file to include in your submission.

A common practice with regards to model saving is to only save models with the best test_loss, rather than checkpoint every epoch. If you want, you can modify the save code to do that instead.

### Model Architecture

In lecture, we introduced the trick to use joint accelerations to compute the next state. You will use your neural network to compute the joint accelerations. This will take in your joint angles, joint velocities, and actions applied to the arm and output joint acceleration values for each link. Then you can use your knowledge of analytical kinematics to derive the joint positions and velocities in the next time step given the joint acceleration values.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
import torch
import torch.nn as nn
import argparse
import time
import pickle
np.set_printoptions(suppress=True)


class DynamicDataset(Dataset):
    def __init__(self, data_file):
        data = pickle.load(open(data_file, "rb" ))
        # X: (N, 6), Y: (N, 4)
        self.X = data['X'].astype(np.float32)
        self.Y = data['Y'].astype(np.float32)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


class Net(nn.Module):
    # ---
    # Your code goes here
    pass
    # ---


def train(model):
    model.train()

    # ---
    # Your code goes here
    # ---


def test(model):
    model.eval()

    # --
    # Your code goes here
    test_loss = 0
    # ---

    return test_loss


# The ratio of the dataset used for testing
split = 0.2

# Do NOT change
# We are only using CPU, and GPU is not allowed.
device = torch.device("cpu")

dataset = DynamicDataset('data_pt2.pkl')
dataset_size = len(dataset)
test_size = int(np.floor(split * dataset_size))
train_size = dataset_size - test_size
train_set, test_set = random_split(dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_set, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=True)

# The name of the directory to save all the checkpoints
timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
model_dir = os.path.join('models', timestr)

# Keep track of the checkpoint with the smallest test loss and save in model_path
model_path = None

epochs = 50
for epoch in range(1, 1 + epochs):
    # ---
    # Your code goes here
    # ---
    pass


## Prediction

After you are done with training, you need to complete the cell below to load the saved checkpoint (in function init_model) and then use it to predict the new state given the current state and action (in function dynamics_step). Please do not modify the arguments to those functions, even though you might not use all of them.

In [ ]:
from arm_dynamics_base import ArmDynamicsBase

class ArmDynamicsStudent(ArmDynamicsBase):
    def init_model(self, model_path, num_links, time_step, device):
        # ---
        # Your code hoes here
        # Initialize the model loading the saved model from provided model_path
        self.model = None
        # ---
        self.model_loaded = True

    def dynamics_step(self, state, action, dt):
        if self.model_loaded:
            # ---
            # Your code goes here
            # Use the loaded model to predict new state given the current state and action
            # Output should be an array of shape (4,1)
            new_state = None
            return new_state
            # ---
        else:
            return state

## Evaluation and Grading

The total number of points for this part is 5. There are 3 types of tests. Your grade for this part will be the average score across each type of test

**For each type, there are 50 tests.** For each test, you get a score of 1, 0.5, or 0. Your final grade for each type is the averaged score across 50 tests * 5.

- *Type 1*: for each test, a constant torque randomly sampled from [-1.5Nm, 1.5Nm] is applied to the first joint of the arm for 5 seconds. If the MSE (Mean Squred Error) between the predicted arm state (arm_student) and the ground truth arm state (arm_teacher) is < 0.0005, you get score 1 for this test. If 0.0005 <= MSE < 0.008, you get score 0.5 for this test. Otherwise you get 0.
- *Type 2*: for each test, a torque that linearly increases from 0 to a random torque in [0.5Nm, 1.5Nm] is applied to the first joint of the arm for 5 seconds. If MSE < 0.0005, you get score 1 for this test. If 0.0005 <= MSE < 0.008, you get score 0.5 for this test. Otherwise you get 0.
- *Type 3*: for each test, one torque is applied for the first 2.5 seconds and another torque is applied for the remaining 2.5 seconds. Both torques are sampled from [-1Nm, 1Nm]. If MSE < 0.015, you get score 1 for this test. If 0.015 <= MSE < 0.05, you get score 0.5 for this test. Otherwise you get 0.


In [ ]:
# DO NOT CHANGE
# Set up grading


import importlib
import score
importlib.reload(score)

# Make sure model_path is correctly set
print(model_path)

# Create the teacher arm
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01
)
arm_teacher = Robot(dynamics_teacher)

# Create the student arm
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01
)
if model_path is not None:
  dynamics_student.init_model(model_path, num_links=2, time_step=0.01, device=torch.device('cpu'))
arm_student = Robot(dynamics_student)

In [ ]:
# DO NOT CHANGE

# Test on randomly sampled torques from [-1.5, 1.5]
score.score_random_torque(arm_teacher, arm_student, gui=False)

In [ ]:
# DO NOT CHANGE

# Test on torques that linearly increase from 0 to a random number from [0.5, 1.5]
score.score_linear_torques(arm_teacher, arm_student, gui=False)

In [ ]:
# DO NOT CHANGE

# Test on one torque applied to the first 2.5s and another torque applied to the second 2.5s
# Both torques are sampled from [-1, 1]
score.score_two_torques(arm_teacher, arm_student, gui=False)

# Part 3: MPC Controller with Learned Dynamics Model

## Part 3.1: Model Architecture

We have a base class Model and a subclass for the 2-link arm. The class Model is a base class for our models. In compute_next_state() method, you have to use the trick to use joint accelerations to compute the next state similar to what you did in Part 2.

In the `Model2Link` class you will use a neural network to compute the joint accelerations by implementing `compute_qddot()` method. This will take 6 values (2 joint angles, 2 joint velocities and 2 actions applied to the arm) and output 2 joint acceleration values

Do not change the arguments for the `__init__()` method even if you do not use them.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
	def __init__(self, num_links, time_step):
		super().__init__()
		self.num_links = num_links
		self.time_step = time_step

	def forward(self, x):
		qddot = self.compute_qddot(x)
		state = x[:, :2*self.num_links]
		next_state = self.compute_next_state(state, qddot)
		return next_state

	def compute_next_state(self, state, qddot):

		# Your code goes here

		return None

	def compute_qddot(self, x):
		pass

class Model2Link(Model):
  def __init__(self, time_step):
    super().__init__(2, time_step)
    # Your code goes here

  def compute_qddot(self, x):
    # Your code goes here
    pass

## Part 3.2: Collect Data
Similar to part 2, we will collect data which will be used to learn a forward model for our 2 link robot arm. Once we have learnt a forward model you will be evaluated on your MPC Controller that uses the learnt dynamics model instead of the true dynamics.

You can modify the collect_data function or write any of your own functions however you choose to. You will be evaluated on the **2 Link Robot**

In [ ]:
# Teacher arm with 2 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

def collect_data(arm):

  # ---
  # You code goes here. Replace the X, and Y by your collected data
  # Control the arm to collect a dataset for training the forward dynamics.
  num_samples = 10000
  X = np.zeros((num_samples, arm.dynamics.get_state_dim() + arm.dynamics.get_action_dim()))
  Y = np.zeros((num_samples, arm.dynamics.get_state_dim()))

  # ---

  return X, Y


In [ ]:
import pickle

# Call the function you have defined above to collect data
X, Y = collect_data(arm)
save_dir = 'dataset'
if not os.path.exists(save_dir):
  os.makedirs(save_dir)

# Save the collected data in the data_pt3.pkl file
data = {'X': X, 'Y': Y}
pickle.dump(data, open(os.path.join(save_dir, 'data_pt3.pkl'), "wb" ))

## Part 3.3: Training the forward model
By now you would be familiar with the basic skeleton of training a forward model.

The starter code already creates the dataset class and loads the dataset with a random 0.8/0.2 train/test split for you. This script should save the model that it trains. You should use a specific procedure for saving, outlined below.

In machine learning, it is a very good practice to save not only the final model but also the checkpoints. Our starter code already configures save_dir for you and for each epoch of your training, you should use the following code to save the model at that epoch.

```
model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
if not os.path.exists(os.path.join(args.save_dir, model_folder_name)):
    os.makedirs(os.path.join(args.save_dir, model_folder_name))
torch.save(model.state_dict(), os.path.join(args.save_dir, model_folder_name, 'dynamics_pt3.pth'))
print(f'model saved to {os.path.join(args.save_dir, model_folder_name, "dynamics_pt3.pth")}\n')
```
The output from running this code should be a folder as below:

```
models/
    2021-03-24_23-57-50/
        epoch_0001_loss_0.00032930/
            dynamics_pt3.pth
        epoch_0002_loss_0.00009413/
            dynamics_pt3.pth   
        ...  
```
You can implement the functions below as you please to collect data

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import pickle
import torch.optim as optim
import argparse
import time

class DynamicDataset(Dataset):
  def __init__(self, datafile):
    data = pickle.load(open(datafile, 'rb'))
    # X: (N, 6), Y: (N, 4)
    self.X = data['X'].astype(np.float32)
    self.Y = data['Y'].astype(np.float32)

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]


def train_one_epoch(model):
  model.train()
	# ---
	# Your code goes here
	# ---
  pass

def test(model):
	model.eval()
	# --
	# Your code goes here
  # --
	test_loss = 0
	return test_loss

def train_forward_model():

	# --
	# Implement this function
  # --

  # Keep track of the checkpoint with the smallest test loss and save in model_path
  model_path = None
  max_test_loss = 1e4
  model = Model2Link(0.01)

  datafile = 'dataset/data_pt3.pkl'
  split = 0.2
  dataset = DynamicDataset(datafile)
  dataset_size = len(dataset)
  test_size = int(np.floor(split * dataset_size))
  train_size = dataset_size - test_size
  train_set, test_set = random_split(dataset, [train_size, test_size])

  train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)

  # The name of the directory to save all the checkpoints
  timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
  model_dir = os.path.join('models', timestr)

  epochs=50
  for epoch in range(1, 1 + epochs):
    # --
    # Your code goes here
    # --
    pass

  return model_path

In [ ]:
model_path = train_forward_model()

## Part 3.4: Completing ArmDynamicsStudent

After you are done with training, you need to complete ArmDynamicsStudent class following the comments below to load the saved checkpoint (in function init_model) and then use it to predict the new state given the current state and action (in function dynamics_step). Please do not modify the arguments to those functions, even though you might not use all of them.

In [ ]:
from arm_dynamics_base import ArmDynamicsBase

class ArmDynamicsStudent(ArmDynamicsBase):
    def init_model(self, model_path, num_links, time_step, device):
        # ---
        # Your code hoes here
        # Initialize the model loading the saved model from provided model_path
        self.model = None
        # ---
        self.model_loaded = True

    def dynamics_step(self, state, action, dt):
        if self.model_loaded:
            # ---
            # Your code goes here
            # Use the loaded model to predict new state given the current state and action
            new_state = None
            return new_state
            # ---
        else:
            return state

## Manually Testing the MPC Controller with the learnt dynamics model
We will now use the learnt dynamics model that you have trained. The model is loaded in the dynamics.init_model method. You can modify the goal positions to see how well is the controller performing similar to what you did before. Feel free to play around with the code in this cell to test your performance before the grading part.


In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False
action = np.zeros((arm.dynamics.get_action_dim(), 1))
if gui:
  renderer = Renderer()
  time.sleep(1)

# Controller
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
device = torch.device('cpu')

# model_path should have the path to the best model that you have trained so far
# which you would like to use for testing the controller
model_path = None
dynamics_student.init_model(model_path, 2, 0.01, device)

# Control loop
action = np.zeros((arm.dynamics.get_action_dim(), 1))
goal = np.zeros((2, 1))
goal[0, 0] = 2.7
goal[1, 0] = 0.5
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()
    action = controller.compute_action(dynamics_student, state, goal, action)
    arm.set_action(action)


## Grading and Evaluation of Part 3
You will be evaluated on how well your controller+learnt dynamics works together. The scoring functions consists of 16 random test goals all of which will be below the x axis and between 0.05 to 1.95 lengths away from the origin.
The controller will call the compute_action method from your MPC class and apply the action for 10 timesteps
```
action = controller.compute_action(dynamics_student, state, goal, action)
```

Each test will run the robot arm for **2.5 seconds**. At the end of the 2.5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.3 and vel_ee < 0.5`

After all of the tests are complete, your score is summed up and then scaled out of 5 total points. You need 15 out of the 16 tests to succeed to get a full score.

In [ ]:
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
model_path = 'dynamics_pt3.pth'
gui=False

In [ ]:
# DO NOT CHANGE
score_mpc_learnt_dynamics(controller, dynamics_student, model_path, gui)

Time Limit
*   MPC evaluation (Part 1): 30 minutes
*   Data Collection with MPC (Part 2): 1 hour 20 minutes
*   Training the forward dynamics model (Part 3): 40 minutes

Note: Time limits are general guidelines. While your code must run within the allotted time, you should be able to achieve full score with a much shorter runtime


Hints and Suggestions:
1. You can use your MPC Controller in your data collection to gather better training samples
2. A good cost function to evaluate your trajectory in MPC is very important and you can use both distance and velocity metrics to define the cost function.
3. As mentioned in the lecture, a constant torque with pseudo gradients seems to work well for this project. You can also use multiple delta values to gather more trajectories to choose from.
4. Since we are passing the MPC object to the controller you can instantiate the MPC class with different parameters like the planning horizon, delta values etc.
5. To speed up data collection, avoid using np.concatenate(), np.stack() or np.append() like functions on your X and Y arrays. Instead, initialize X and Y arrays with all zeros using the correct shape and then fill in the values one by one. This is much faster in numpy. Using Ray is optional

# Other Requirements and Hints

Time Limit
*   MPC evaluation (Part 1): 30 minutes
*   Learned Forward Model (Part 2): Less than a hundred epochs of training (<= 25 mins) should suffice for achieving the full points. Again, the shorter your model training time is the better.
*   Data Collection with MPC (Part 3): 1 hour 20 minutes
*   Training the forward dynamics model (Part 3): 40 minutes

Note: Time limits are general guidelines. While your code must run within the allotted time, you should be able to achieve full score with a much shorter runtime


Hints and Suggestions:
1. Choosing the right policy to collect datasets for this project (parts 2 and 3) is important. You need to think about how to do it properly so that your trained model will pass the tests successfully. It is in general very hard to learn the ground truth forward dynamics completely (that works for any distribution of actions), and during testing small errors can accumulate, leading to drastic failure in the end. You might want to try overfitting on the test cases to begin with.
2. Make sure that your dataset for part 2 is less than 100 Mb, which is pretty much sufficient for achieving full marks. Collecting datasets can be time-consuming and you could parallelize this process for some speed-up using [ray](https://www.ray.io/). Make sure your data collection in part 2 takes <= 25 mins.
3. You can use your MPC Controller in your data collection for part 3 to gather better training samples
4. A good cost function to evaluate your trajectory in MPC is very important and you can use both distance and velocity metrics to define the cost function.
5. As mentioned in the lecture, a constant torque with pseudo gradients seems to work well for this project. You can also use multiple delta values to gather more trajectories to choose from.
6. Since we are passing the MPC object to the controller you can instantiate the MPC class with different parameters like the planning horizon, delta values etc.
7. To speed up data collection, avoid using np.concatenate(), np.stack() or np.append() like functions on your X and Y arrays. Instead, initialize X and Y arrays with all zeros using the correct shape and then fill in the values one by one. This is much faster in numpy. Using Ray is optional
8. You may reuse your part 2 model for part 3, but it may be beneficial to re-collect data to train a model specific to part 3.

General Tips and Requirements
- NO GPU: No GPU is required or allowed for this assignment and we will test your code without GPUs.
- Loss Function: This is essentially a regression problem so think about what losses are suitable for regression.
- Optimizer: While it is possible to use a simple optimizer to achieve the desired accuracy, the training time can be quite high. There exists a number of optimizers implemented in PyTorch that have much faster convergence.
- Seeding. Please use seeding in your code to make sure your results are reproducible.